In [ ]:
#Load SAHI detection output-- COCO format saved as CSV
import pandas as pd

df = pd.read_csv("annotations.csv")
df.head()

In [ ]:
#Calculate distance from each COCO bounding box to image center

# Resolution in cm/px
resolution_cm_px = 1.14 #specify according to your image set

# Convert resolution from cm/px to meters/px
resolution_m_px = resolution_cm_px / 100.0

# Image center in pixels
center_x_px, center_y_px = 2736, 1824 #Specify according to your image set

# Calculate the center point in meters
center_x_m = center_x_px * resolution_m_px
center_y_m = center_y_px * resolution_m_px

# Function to calculate distance from center
def calculate_distance_from_center(row):
    # Get the coordinates of the bounding box (x, y, width, height)
    x, y, width, height = row['bbox']

    # Calculate the center point of the bounding box in pixels
    bbox_center_x_px = x + (width / 2)
    bbox_center_y_px = y + (height / 2)

    # Calculate the center point of the bounding box in meters
    bbox_center_x_m = bbox_center_x_px * resolution_m_px
    bbox_center_y_m = bbox_center_y_px * resolution_m_px

    # Calculate the distance from the center of the image in meters
    distance_m = ((center_x_m - bbox_center_x_m)**2 + (center_y_m - bbox_center_y_m)**2)**0.5

    return distance_m

# Apply the function to calculate distance to each row in the DataFrame
df['distance_to_center_m'] = df.apply(calculate_distance_from_center, axis=1)

df.head()

In [ ]:
#Export as CSV
savepath = "" #Desired output directory
df.to_csv(savepath+"distance_anns.csv") #desired filename